In [1]:
import gc
import pathlib
import sys

In [2]:
import numpy as np
import joblib
import pandas as pd
from tqdm import tqdm

In [3]:
root_dir_path = pathlib.Path("..")
data_dir_path = root_dir_path / "data"
interim_dir_path = data_dir_path / "interim"
train_path = interim_dir_path / "train.parquet"
test_path = interim_dir_path / "test.parquet"
models_dir_path = root_dir_path / "models"
model_path = models_dir_path / "model.joblib"
prediction_path = models_dir_path / "prediction.parquet"
src_dir_path = root_dir_path / "src"

In [4]:
sys.path.append(str(src_dir_path))

In [5]:
from package.constants import *
from package.feature_extraction import *

In [6]:
max_lags = 28

In [7]:
train = pd.read_parquet(train_path)
test = pd.read_parquet(test_path)

In [8]:
dates = test["date"].unique()
data = pd.concat([train, test])

In [9]:
del train

In [10]:
gc.collect()

40

In [11]:
model = joblib.load(model_path)

In [12]:
for date in tqdm(dates):
    is_date = (data["date"] >= date - np.timedelta64(max_lags, "D")) & (
        data["date"] <= date
    )
    intermediate = extract_features(data[is_date])

    is_date = intermediate["date"] == date
    X = intermediate.loc[is_date, features]
    y_pred = model.predict(X)
    y_pred = np.maximum(0.0, y_pred)

    is_date = test["date"] == date
    test.loc[is_date, "demand"] = y_pred

100%|██████████| 56/56 [1:18:21<00:00, 83.96s/it]


In [13]:
test.to_parquet(prediction_path)